In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("tree").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/27 20:10:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
data = spark.read.csv("College.csv", inferSchema = True, header = True)

In [4]:
data.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [5]:
data.head(1)

[Row(School='Abilene Christian University', Private='Yes', Apps=1660, Accept=1232, Enroll=721, Top10perc=23, Top25perc=52, F_Undergrad=2885, P_Undergrad=537, Outstate=7440, Room_Board=3300, Books=450, Personal=2200, PhD=70, Terminal=78, S_F_Ratio=18.1, perc_alumni=12, Expend=7041, Grad_Rate=60)]

In [6]:
from pyspark.ml.feature import VectorAssembler

In [7]:
data.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [8]:
assembler = VectorAssembler(inputCols = ["Apps",
 "Accept",
 "Enroll",
 "Top10perc",
 "Top25perc",
 "F_Undergrad",
 "P_Undergrad",
 "Outstate",
 "Room_Board",
 "Books",
 "Personal",
 "PhD",
 "Terminal",
 "S_F_Ratio",
 "perc_alumni",
 "Expend",
 "Grad_Rate"], outputCol = "features" )

In [9]:
output = assembler.transform(data)

In [10]:
from pyspark.ml.feature import StringIndexer

In [11]:
indexer = StringIndexer(inputCol = "Private", outputCol = "PrivateIndex")

In [12]:
output_fixed = indexer.fit(output).transform(output)

In [13]:
output_fixed.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- PrivateIndex: double (nullable = false)



In [14]:
final_data = output_fixed.select("features", "PrivateIndex")

In [15]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [16]:
from pyspark.ml.classification import DecisionTreeClassifier, GBTClassifier, RandomForestClassifier

In [17]:
from pyspark.ml import Pipeline

In [20]:
dtc = DecisionTreeClassifier(labelCol = "PrivateIndex", featuresCol = "features")
rfc = RandomForestClassifier(labelCol = "PrivateIndex", featuresCol = "features")
gbt = GBTClassifier(labelCol = "PrivateIndex", featuresCol = "features")

In [21]:
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

In [23]:
dtc_preds = dtc_model.transform(test_data)
rfc_preds = rfc_model.transform(test_data)
gbt_preds = gbt_model.transform(test_data)

In [24]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [25]:
my_binary_eval = BinaryClassificationEvaluator(labelCol = "PrivateIndex")

In [26]:
print("DTC")
print(my_binary_eval.evaluate(dtc_preds))

DTC
0.799863387978142


In [27]:
print("RFC")
print(my_binary_eval.evaluate(rfc_preds))

RFC
0.9735883424408016


In [29]:
rfc_preds.printSchema()

root
 |-- features: vector (nullable = true)
 |-- PrivateIndex: double (nullable = false)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [30]:
my_binary_eval2 = BinaryClassificationEvaluator(labelCol = "PrivateIndex", rawPredictionCol = "prediction") 

In [31]:
gbt_preds.printSchema()

root
 |-- features: vector (nullable = true)
 |-- PrivateIndex: double (nullable = false)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [33]:
print("GBT")
print(my_binary_eval2.evaluate(gbt_preds))

GBT
0.8498975409836066


22/04/27 20:35:08 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/04/27 20:35:08 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


In [34]:
rfc = RandomForestClassifier(numTrees = 150, labelCol = "PrivateIndex", featuresCol = "features")

In [35]:
rfc_model = rfc.fit(train_data)

In [38]:
rfc_preds = rfc_model.transform(test_data)

In [39]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [40]:
my_binqry_eval = BinaryClassificationEvaluator(labelCol = "PrivateIndex")

In [43]:
print("RFC")
print(my_binary_eval.evaluate(rfc_preds))

RFC
0.9764344262295083


In [45]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [47]:
acc_eval = MulticlassClassificationEvaluator(labelCol = "PrivateIndex", metricName = "accuracy")

In [48]:
rfc_acc = acc_eval.evaluate(rfc_preds)

In [49]:
rfc_acc

0.9393939393939394